# Tension Lab Analysis Code ヽ(•‿•)ノ

## Define The Sample Parameters
#### You will need to change these for every sample type

In [ ]:
thickness = 6.2 # Input thickness in mm
width = 12.3 # Input Width in mm
Area = thickness*width #in mm^2
#print("Area =", Area, "mm^2" )

## Read The Data File
#### We will be using Pandas as our data analysis library

In [ ]:
import pandas as pd

fileName = '6061 Aluminum Sample 1.csv'

Data = pd.read_csv(fileName) #this function reads the .csv file

#You can access the keys to get a list of the column names
dKeys = Data.keys() #This is the name of all the columns of Data
#print(dKeys)
#print(Data[dKeys[3]]) #This accesses one of the data keys if you don't know the names
#print(Data['Axial Strain (mm/mm)']) #This accesses one of the data keys if you do know the names

#You can also print out parts of the file
#Data.head() #run this to output the first 5 lines of data
#Data.tail() #run this instead to output the last 5 lines of data

#And get help on the function if you're confused
#help(pd.read_csv)

## Add Extra Columns to the Data
#### We can directly add items to a pandas data type using dictName['new item']= *something*

In [ ]:
emptyList = [[]]*len(Data.index) #This is an empty list with the same length as the data file
#Note, when you multiply Python lists it just copies the list, e.g. [[1]]*3=[[1],[1],[1]]

#Here's some example new dictionary calculations
Data['Strain (mm/mm)'] = Data['Axial Strain (mm/mm)'] #this adds another strain column to the data
Data['Stress (MPa)'] = Data['Load (N)']/Area #this adds stress to the data
    
#YOU'LL NEED TO ADD FUNCTIONS HERE
Data['Instantaneous Area (mm^2)'] = emptyList #Calculate the instantaneous area using the original dimensions and the transverse strain
Data['True Stress (MPa)'] = emptyList #Add in the true stress here
Data['True Strain (mm/mm)'] = emptyList #Add in the true strain here,
#Data.head()

## Plot The Data
#### We will use zoomed sections to better visualize the elastic region

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(10,7))

#Plot the stress/strain in the first column
axs[0,0].plot(Data['Strain (mm/mm)'],Data['Stress (MPa)'])
axs[1,0].plot(Data['Strain (mm/mm)'],Data['Stress (MPa)'])
###You can modify this for true stress and true strain###

#Plot the axial/transverse strain in the second column
axs[0,1].plot(Data['Strain (mm/mm)'],Data['Transverse Strain (mm/mm)'])
axs[1,1].plot(Data['Strain (mm/mm)'],Data['Transverse Strain (mm/mm)'])

#Zoom in on the data in the bottom plots
eZoom = 0.01; tZoom = 0.004; sZoom = 350
axs[0,0].plot([0,0,eZoom,eZoom,0],[0,sZoom,sZoom,0,0],'r--') #superimpose the zoom box
axs[0,1].plot([0,0,eZoom,eZoom,0],[0,tZoom,tZoom,0,0],'r--') #superimpose the zoom box
axs[0,0].set_xlim(left = 0)
axs[0,0].set_ylim(bottom = 0)
axs[0,1].set_xlim(left = 0)
axs[0,1].set_ylim(bottom = 0)
axs[1,0].set_xlim(left = 0, right = eZoom) # Zoom strain
axs[1,0].set_ylim(bottom = 0, top = sZoom) #Zoom stress
axs[1,1].set_xlim(left = 0, right = eZoom) # Zoom strain
axs[1,1].set_ylim(bottom = 0, top = tZoom) #Zoom transverse strain

#Add Labels
axs[0,0].set_title("Engineering Stress vs Strain")
axs[0,1].set_title("Axial vs Transverse Strain")
axs[0,0].set_xlabel('Strain (mm/mm)')
axs[0,1].set_xlabel('Strain (mm/mm)')
axs[1,0].set_xlabel('Strain (mm/mm)')
axs[1,1].set_xlabel('Strain (mm/mm)')
axs[0,0].set_ylabel('Stress (MPa)')
axs[1,0].set_ylabel('Stress (MPa)')
axs[0,1].set_ylabel('Transverse Strain (mm/mm)')
axs[1,1].set_ylabel('Transverse Strain (mm/mm)')
plt.tight_layout()
plt.show()

## Getting Material Properties 
#### Young's modulus is the slope of stress vs strain in elastic region.
#### Poisson's ratio is the ratio between axial and transverse strain in the elastic region.
#### Yield strength is the point where the material starts to plastically deform.
#### There are many ways to do these calculations, the code here is a manual method.

In [ ]:
from scipy.stats import linregress #This is a linear regression function built into the Scipy library. 

def modulusFit(Strain,Stress,a,b):
    '''This is a linear fit to data between the data indices for a and b. Note, this will
    return an error if a or b are outside the length of Strain and Stress.'''
    
    #Fit the modulus
    E,C,R,P,Err = linregress(Strain[a:b],Stress[a:b]) #The data outputs the slope (E), intercept (C), regression (R) value, P-value and standard error
    #Note: Python lets you save multivariable outputs with a comma, i.e. a,b=[1,2] will give a=1 and b=2
    
    #Make a line for the fit data
    Y = [0.0, max(Stress)] #this is a list of length 2 for plotting the fit data later
    X = [(y-C)/E for y in Y] #these are points that you can plot to visualize the data being fit, inverted from y=E*x+C, x=(y-C)/E
    return E,C,R,X,Y

def yieldStress(Strain,Stress,E,C,eOffset=0.002):
    '''This function will find the yield stress based on a 0.2% offset strain method.
    You must input the stress, strain, Youngs modulus E, fit line intercept point b, and
    can change the strain offset value.'''
    
    yP = next((i for i,x in enumerate(Strain) if Stress[i] < E*(x-eOffset) + C),len(Strain)-1)
    #This code finds the first point where the Stress exceeds the strain offset line defined by y=E*(x-eOffset)+b
    #This is the simplest way to determine a slope intercept, but it only works if the stress and the offset line intersect
    
    return yP #this returns the index i of the yield stress, if you want it to return the stress, use: return Stress[yP]

def PoissonFit(axialStrain,transverseStrain,a,b):
    '''This is a linear fit to data between the data indices for a and b. Note, this will
    return an error if a or b are outside the length of Strain.'''
    
    #Fit the modulus
    nu,C,R,P,Err = linregress(axialStrain[a:b],transverseStrain[a:b]) #The data outputs the slope (nu), intercept (C), regression (R) value, P-value and standard error
    #Note: Python lets you save multivariable outputs with a comma, i.e. a,b=[1,2] will give a=1 and b=2
    
    #Make a line for the fit data
    yEnd = min([int(1.5*b),len(transverseStrain)-1])
    Y = [0.0, transverseStrain[yEnd]]
    X = [(y-C)/nu for y in Y] #these are points that you can plot to visualize the data being fit, inverted from y=nu*x+C, x=(y-C)/nu
    return nu,R,X,Y

## Use the functions to find material properties
## YOU WILL NEED TO EDIT THESE FITTING POINTS TO DETERMINE MATERIAL PROPERTIES

In [ ]:
#Save dummy variables to make the code cleaner below
aStrain = Data['Strain (mm/mm)'].values[:-1]
tStrain = Data['Transverse Strain (mm/mm)'].values[:-1]
aStress = Data['Stress (MPa)'].values[:-1]

## Young's Modulus Fit
aE = 20; bE = 150 #You will need to edit these values to determine the best fit
E,C,RE,XE,YE = modulusFit(aStrain,aStress,aE,bE)

## Yield Strength Fit
eOff = 0.002 #0.2% strain offset
iYield = yieldStress(aStrain,aStress,E,C,eOffset=eOff) #Note, this is the index of the yield
sy = aStress[iYield]

## Poisson's Ratio Fit
anu = 100; bnu = 300  #You will need to edit these values to determine the best fit
nu,Rnu,Xnu,Ynu = PoissonFit(aStrain,tStrain,anu,bnu)

## See how the fits look and adjust accordingly
#### We'll just be plotting the zoomed region here so we can see the elastic region better

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10,5))

axs[0].plot(aStrain,aStress) #Plot the stress/strain in the first column
axs[1].plot(aStrain,tStrain) #Plot the axial/transverse strain in the second column

#Zoom in on the data in the bottom plots
eZoom = 0.01; tZoom = 0.004; sZoom = 350
axs[0].set_xlim(left = 0, right = eZoom) # Zoom strain
axs[0].set_ylim(bottom = 0, top = sZoom) #Zoom stress
axs[1].set_xlim(left = 0, right = eZoom) # Zoom strain
axs[1].set_ylim(bottom = 0, top = tZoom) #Zoom transverse strain

#Plot the Young's fits
axs[0].plot(aStrain[aE],aStress[aE],'rd') #This is the first point we're fitting from 
axs[0].plot(aStrain[bE],aStress[bE],'rs') #this is the last point we're fitting to
# axs[0].plot(aStrain[aE:bE],aStress[aE:bE],'r.') #this will show all the data we're fitting
axs[0].plot(XE,YE,label='E='+str(round(E*1e-3,1))+' GPa, R='+str(round(RE,4)))

#Plot the Yield Stress
xOffset = [x+eOff for x in XE]
axs[0].plot(aStrain[iYield],aStress[iYield],'ro',label=r'Yield, $\sigma_y$='+str(round(aStress[iYield],1))+' MPa')
axs[0].plot(xOffset,YE,'--')

#Plot the Poisson's fit
axs[1].plot(aStrain[anu],tStrain[anu],'rd') #This is the first point we're fitting from
axs[1].plot(aStrain[bnu],tStrain[bnu],'bs') #This is the last point we're fitting from
# axs[1].plot(aStrain[anu:bnu],tStrain[anu:bnu],'b.') #These are all the points we're fitting
axs[1].plot(Xnu,Ynu,label=r"Poisson's Ratio Fit, $\nu$="+str(round(nu,3)))

#Add Labels
axs[0].set_title("Modulus Fit")
axs[1].set_title("Poisson's Fit")
axs[0].set_xlabel('Strain (mm/mm)')
axs[1].set_xlabel('Strain (mm/mm)')
axs[0].set_ylabel('Stress (MPa)')
axs[1].set_ylabel('Transverse Strain (mm/mm)')
axs[0].legend()
axs[1].legend()
plt.tight_layout()
plt.show()

#### You can see that slightly different 'linear regions' for measuring the slope can produce noticeably different results

## Average the Data
#### Once you have found good fitting points, you will need to save these results and average them for all the data

In [ ]:
from numpy import mean,std 

#You can call the mean and std (standard deviation) for your data
youngsModuli = [] #Add Data Here
poissonsRatios = [] #Add Data Here
yieldStrengths = [] #Add Data Here

#Calculate Averages
averageE = mean(youngsModuli)
stdE = std(youngsModuli)
averagenu = mean(poissonsRatios)
stdnu = std(poissonsRatios)
averageSy = mean(yieldStrengths)
stdSy = std(yieldStrengths)

#### Brief aside - here's a quick look at how the yield function works

In [ ]:
aList = [1,3,5,2,1,6,8,7] #this is a list of integers
aListSquared = [x**2 for x in aList] #this is a list comprehension that squares every entry in aList, it will go through the list elements sequentially
aListIndex = [i for i,x in enumerate(aList)] #this spits out all the indices of the elements in a list, here just 0,1,2,3...
aListPart = [x for x in aList if x>4] #this finds all the list elements that are greater than 4
aListNext = next(x for x in aList if x>4) #this finds the next list element that is greater than 4
aListNextIndex = next(i for i,x in enumerate(aList) if x>4) #this finds the index of the next list value greater than 4
print(aList)
print(aListSquared)
print(aListIndex)
print(aListPart)
print(aListNext)
print(aListNextIndex)
print(aList[aListNextIndex])